# Cell-cell communication

*TL;DR We provide a brief overview of basic concepts and assumtions for cell-cell communication infrence in single-cell transcritpomics data, we provide examples of the two most common approaches for CCC inference: those that focus on ligand-receptor interactions and those that include downstream response.*

### Background

Cell communication is the process of cells to react to stimuli from their environment, and also from themselves. In mutlicellular organisms, the dynamic coordination of cells, also called cell-cell communication (CCC), is involved in many biological processes, such as apoptosis and cell migration, and is consequently essential in homeostasis, development, and disease. CCC commonly focuses on protein-mediated interactions, most typically percieved as a secreted ligand binding to its corresponding plasma membrane receptor. However, this picture can be broadened to include secreted enzymes, extra-cellular matrix proteins, transporters, and interactions that require the physical contact between cells, such as cell-cell adhesion proteins and gap junctions (Armingol et al., 2021). Cell communication is not independent of other processes, but the contrary, as external stimuli commonly elicit a downstream response. In the case of CCC, this is typically percieved as the induction of canonical pathways and downstream transcription factors in the cells receiving the signal, or receiver cells. Ultimately these exernal stimuli alter the function of receiver cells, and this alteration is further propagated via the subsequent interaction of these cells with their microenvironment. 

Traditionally, the study of CCC required extensive domain knowledge and specialized in-situ biochemical assays, such proximity labelling proteomics, co-immunoprecipitation, and yeast two-hybrid screening (Armingol et al., 2021). Yet, the rapid developments and drop in costs in transcriptomics data generation has enabled a paradigm shift away from focusing on what type of cells and cell groups are present, but rather on the relationships betweeen them (Almet et al.). As a consequence, CCC inference from single-cell data is now becoming a routine approach, capable of providing a system-level hypotheses of intercellular crosstalk in vivo.

### Approaches

As a result of this increased interest, a number of computational tools for CCC inference from single-cell transcritpomics have emerged that can be classified as those that predict CCC interactions alone, commonly referred as ligand-receptor inference methods (e.g. CellPhoneDB, CellChat, Connectome), and those that additionally estimate intracellular activities induced by CCC (e.g. SoptSC, NicheNet, CytoTalk). Both categories of tools use gene expression information as a proxy of protein abundance, and typically require the clustering of cells into biologically-meaningful groups (See Annotation tutorial). These CCC tools infer intercellular crosstalk between pairs of cell groups, one group being the source and the other the receiver of a CCC event. CCC events are thus commonly represented as protein-protein interactions, expressed by the source and receiver cell clusters.

The information about the interacting proteins is commonly extracted from prior knowledge resources. In the case of ligand-receptor methods, the interactions can also be represented by heteromeric protein complexes, as different subunit combinations can induce distinct responses and the inclusion of protein complex information has been shown to reduce false positive rates (Efremova, Jin). On the other hand, the approaches that model intracellular signalling also leverage the functional information in receiver cell types, and thus require information of intracellular protein-protein interaction newtorks and gene regulatory interactions (NicheNet).

Recent work has shown that the choice of method and/or resource leads to limited agreement in inferred predictions (Dimitrov et al; Wang, et al; Liu, Sun, and Wang), thus prompting caution when interpreting their output. The CCC field is further plagued by the lack of ground truth (Armingol, Almet), capable of capturing the complex and dynamic interplay between large numbers of cells and molecules. Nevertheless, independent evaluations have shown that CCC methods are fairly robust to the introduction of noise (Jin, Dimitrov, Wang), and are largely concordant with alternative data modalities such as intracellular signalling (Dimitrov) and spatial information (Dimitrov; Liu, Sun, and Wang).

Note that here we generalize for the sake of simplicity, as the CCC field contains a plethora of different and newly emerging approaches - we highlight those in the **Outlook** section.

**FIGURE**: Generic figure on CCC /w Ligand-Receptor + Downstream signalling shown

## CONTENT

Remind of the Wang dataset

#### Steady-state ligand-receptor
CPDB, LIANA

CPDB and LIANA run on Condition + standard dotplot of top x interactions

#### NicheNet

#### NicheNet x LIANA (LIANA by sample):

In contrast to LIANA, NicheNet aims to deepen in the intra-cellular mechanisms that connect a ligand with a set of transcriptional targets, making an extensive usage of prior knowledge from multiple sources. LIANA and NicheNet are not mutually exclusive, but in certain scenarios could be rather complementary, given that they aim to explore different aspects of inter- and intra-cellular communication.

Robin:

NicheNet and ligand-receptor methods, such as CellPhoneDB, can all be used to
study intercellular communication, but it is important to note that they address different questions.
Basic ligand-receptor  methods
predict communicating pairs of cells and the facilitating ligand-receptor interactions by looking at the
expression of interacting ligands and receptors, typically work on "steady-state" data. On the contrary, NicheNet predicts which of these
inferred ligand-receptor links are possibly the most functional based on gene expression changes that
are induced in the target cell type. 
The typical application of NicheNet is **NOT** “steady state“ data, ecause NicheNet predicts
which ligands influence gene expression in a target cell type, there should be information in
the data about gene expression changes in the target cell type that are possibly caused by
interacting cells (differential expression).

Therefore, the choice of which method to use depends on the type
of data and research question.



https://saezlab.github.io/liana/articles/liana_nichenet.html

https://github.com/saezlab/eccb2022_sc_funcomics/blob/main/2_cell_comm.Rmd#L119

## Key takeaways

#### Assumptions & Limitations:
1. Communication Ranges (Palla, This ligand range thing), e.g. endocrine/long-distance is missing
2. Protein-mediated alone, with expression data as a proxy for protein abundance (extracellular vehicles, small molecules, hormones, etc)
3. Signalling does not occur at the cell type level 
4. Canonical limitations of scRNA-Seq
5. Typically one-to-one events, which is an oversimplification (Erick: "To understand the role of each cell within its local community, one must identify the protein messages passed between cells; measuring expressed messenger molecules and their associated pathways is fundamental to understanding the directionality, magnitude and biological relevance of CCC.")
6. Experimental Validation is required
7. Diffusion etc

LIANA's insights (+ that other one that does spatial benchmarking); disagreement (Dimitrov), complexes (Efremova et al, Jin et al.,), downstream (CytoSig, scSeqComm), spatial (Luz, Rico), etc

#### Outlook:
1. Metbolite (MEBOCOST, CellPhoneDBv4, scConnect)
2. Single-cell resolution (Niches, SoptSC, + this other one)
3. Link to Spatial Methods (Misty, NCEM) + Chapter
4. Across Context (Mention NATMI, Connectome, CellChat), highlight Tensor-Cell2cell)
5. Those that focus on both ligand-receptor and downstream (CellCall, scSeqComm, etc)

## Quiz

- Q1. What are the first three limitations that come to mind when inferring CCC from single-cell transcriptomics data?
- Q2. What is the role of heteromeric complexes in ligand-receptor inference?
- Q3. Can you name an advantage and a disadvantage of inference based on prior-knowledge?

## References

```{bibliography}
:filter: docname in docnames
```